In [1]:
import gzip
import json
import html
import re
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model

In [2]:
dataset_path = 'ratebeer.json.gz'
with gzip.open(dataset_path, 'rt', encoding='utf-8') as f:
    print(f.readline())

{'beer/name': 'John Harvards Simcoe IPA', 'beer/beerId': '63836', 'beer/brewerId': '8481', 'beer/ABV': '5.4', 'beer/style': 'India Pale Ale &#40;IPA&#41;', 'review/appearance': '4/5', 'review/aroma': '6/10', 'review/palate': '3/5', 'review/taste': '6/10', 'review/overall': '13/20', 'review/time': '1157587200', 'review/profileName': 'hopdog', 'review/text': 'On tap at the Springfield, PA location. Poured a deep and cloudy orange (almost a copper) color with a small sized off white head. Aromas or oranges and all around citric. Tastes of oranges, light caramel and a very light grapefruit finish. I too would not believe the 80+ IBUs - I found this one to have a very light bitterness with a medium sweetness to it. Light lacing left on the glass.'}



In [3]:
# dataset_path = 'ratebeer.json.gz'
# allRatings = []
# count = 0
# failCount = 0
# review_text_pattern = r",\s*'review/text'\s*:\s*'[^']*'[,}]\s*"
# review_text_pattern_2 = r",\s*'review/text'\s*:\s*\"[^']*\"[,}]\s*"
# # review_text_pattern_2 = r"(,\s*'review/text'\s*:\s*['\"]([^'\"]*)['\"])"
# # review_text_pattern_2 = r"(,\s*'review/text'\s*:\s*['\"])[^'\"]*(['\"])"
# # review_text_pattern_2 = r"(,\s*'review/text'\s*:\s*['\"])([^'\"]*)(['\"])"
# # review_text_pattern_2 = r"(,\s*'review/text'\s*:\s*['\"])([^'\"]+)(['\"])"

# with gzip.open(dataset_path, 'rt', encoding='utf-8') as f:
#     for line in f:
#         line = line.strip()
#         # line = line.replace('"', "'")
#         line = html.unescape(line)
#         lineog = line
#         line = re.sub(review_text_pattern, '}', line)  # Remove 'review/text' key and its content
#         # line = re.sub(review_text_pattern_2, '}', line)  # Remove 'review/text' key and its content
#         line = line.replace("'", '"')
#         # print(lineog)

#         try:
#             data = json.loads(line)
#             # allRatings.append(data)
            
#         except json.JSONDecodeError as e:
#             print(f"Error decoding JSON: {e}")
#             print(lineog)
#             print(line)
#             failCount += 1
        
#         if count == 10:
#             # print(line)
#             break
#         count += 1

# print(failCount)

In [4]:
dataset_path = 'ratebeer.json.gz'
allRatings = []
count = 0
failCount = 0
review_text_pattern = r",\s*'review/text'\s*:\s*'[^']*'[,}]\s*"

with gzip.open(dataset_path, 'rt', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        line = html.unescape(line)
        line = re.sub(review_text_pattern, '}', line)  # Remove 'review/text' key and its content
        line = line.replace("'", '"')

        try:
            data = json.loads(line)
            allRatings.append(data)
            
        except json.JSONDecodeError as e:
            failCount += 1

        count += 1

print(count, failCount)

2924164 48539


In [5]:
print(allRatings[0])
print(type(allRatings[0]))

{'beer/name': 'John Harvards Simcoe IPA', 'beer/beerId': '63836', 'beer/brewerId': '8481', 'beer/ABV': '5.4', 'beer/style': 'India Pale Ale (IPA)', 'review/appearance': '4/5', 'review/aroma': '6/10', 'review/palate': '3/5', 'review/taste': '6/10', 'review/overall': '13/20', 'review/time': '1157587200', 'review/profileName': 'hopdog'}
<class 'dict'>


In [7]:
trainingRatio = 0.6
validationRatio = 0.7

ratingsTrain = allRatings[:int(len(allRatings)*trainingRatio)]
ratingsValid = allRatings[int(len(allRatings)*trainingRatio) : int(len(allRatings)*validationRatio)]
ratingsTest = allRatings[int(len(allRatings)*validationRatio):]

print(len(ratingsTrain), len(ratingsValid), len(ratingsTest))

1725375 287562 862688


In [9]:
def feature(datum):
    # Calculate the number of times ! is present in the review
    # Return the value to be passed into feature (X)
    feat = [1]
    feat.append(eval(datum['review/appearance']) > 0.5)
    feat.append(eval(datum['review/aroma']) > 0.5)
    feat.append(eval(datum['review/palate']) > 0.5)
    feat.append(eval(datum['review/taste']) > 0.5)
    return feat

In [10]:
X = [feature(r) for r in ratingsTrain]
Y = [eval(r['review/overall']) for r in ratingsTrain]

In [11]:
model = linear_model.LinearRegression(fit_intercept=False)
model.fit(X, Y)

LinearRegression(fit_intercept=False)

In [14]:
def MSE(labels, predictions):
    sse = sum([(p-l)**2 for (p,l) in zip(predictions, labels)])
    return sse / len(predictions)

In [15]:
print(model.coef_)
xValid = [feature(r) for r in ratingsValid]
yValid = [eval(r['review/overall']) for r in ratingsValid]

[0.31936544 0.07553991 0.08921403 0.10037101 0.15186566]


In [16]:
yPred = model.predict(xValid)
mse = MSE(yValid, yPred)
mse

0.012536702999114236

In [17]:
xValid[5]

[1, True, True, True, True]